# Implied volatility for t = 0

No closed form solution for BSM really exists so correct solution gets estiamted (hence, implied)

In [16]:
import math
from scipy import stats
import pandas as pd
from urllib2 import urlretrieve


ImportError: No module named 'urllib2'

In [10]:
def bsm_call_value(S_0, K, T, r, sigma):
    """European Call option model via BSM.
    Nothing exotic or American about this, simple
    bsm w/o optimal execution time etc.
    
    Parameters
    ----------
    S_0: float
        initial stock price
    K: float
        strike price
    T: float
        time to maturity
    r: float
        interest rate
    sigma: float
        implied volatility factor in diffusion term form
    
    Returns
    -----------
    value: float
        calculated value of European Call option
    
    """
    
    s0 = float(S_0)
    d1 = (math.log(s0/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)) # upper bound
    d2 = (math.log(s0/K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)) # lower bound
    return (s0 * stats.norm.cdf(d1, 0., 1.0) - K * math.exp(-r * T) * stats.norm.cdf(d2, 0., 1.0))

In [11]:
def bsm_calc_vega(S_0, K, T, r, sigma):
    """Calculate sensitivity to volatility"""
    s0 = float(S_0)
    d1 = math.log(s0/K) + (r + 0.5 * sigma **2) * T / (sigma * sqrt(T))
    return s0 * stats.norm.cdf(d1, 0.0, 1.0) * sqrt(T) # Vega

In [12]:
help(stats.norm)

Help on norm_gen in module scipy.stats._continuous_distns object:

class norm_gen(scipy.stats._distn_infrastructure.rv_continuous)
 |  A normal continuous random variable.
 |  
 |  The location (loc) keyword specifies the mean.
 |  The scale (scale) keyword specifies the standard deviation.
 |  
 |  %(before_notes)s
 |  
 |  Notes
 |  -----
 |  The probability density function for `norm` is::
 |  
 |      norm.pdf(x) = exp(-x**2/2)/sqrt(2*pi)
 |  
 |  %(after_notes)s
 |  
 |  %(example)s
 |  
 |  Method resolution order:
 |      norm_gen
 |      scipy.stats._distn_infrastructure.rv_continuous
 |      scipy.stats._distn_infrastructure.rv_generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  fit(self, data, **kwds)
 |      Return MLEs for shape, location, and scale parameters from data.
 |      
 |      MLE stands for Maximum Likelihood Estimate.  Starting estimates for
 |      the fit are given by input arguments; for any arguments not provided
 |      with starting es

In [13]:
def bsm_call_imp_vol(S_0, K, T, r, C_0, sigma_est, it=100):
    """Implied volatility calculation of European call option in Black Scholes
    
    Parameters
    -----------
    S_0: float
        initial stock price, at time t=0
    K: float
        strike price
    T: float
        time to maturity, epoch
    r: float
        interest rate
    C_0: float
        initial call price
    sigma_est: float
        estimated implied volatility
    it: int
        number of iterations
    
    Returns
    --------
    
    estimated_sigma: float
        estimated volatility value
    
    """
    
    for i in range(it):
        sigma_est -= ((bsm_call_value(S_0, K, T, r, sigma_est) - C_0) / bsm_calc_vega(S_0, K, T,
                                                                              r, sigma_est))
    return sigma_est


In [30]:
# Run Estimation

h5 = pd.HDFStore('./vstoxx_data_31032014.h5', 'r')
futures = h5['futures_data']
options = h5['options_data']
h5.close()

In [31]:
futures

,DATE,EXP_YEAR,EXP_MONTH,PRICE,MATURITY,TTM
496,1396224000000000000,2014,4,17.85,1397779200000000000,0.049
497,1396224000000000000,2014,5,19.55,1400198400000000000,0.126
498,1396224000000000000,2014,6,19.95,1403222400000000000,0.222
499,1396224000000000000,2014,7,20.40,1405641600000000000,0.299
500,1396224000000000000,2014,8,20.70,1408060800000000000,0.375
501,1396224000000000000,2014,9,20.95,1411084800000000000,0.471
502,1396224000000000000,2014,10,21.05,1413504000000000000,0.548
503,1396224000000000000,2014,11,21.25,1416528000000000000,0.644


In [32]:
futures.describe()

,DATE,EXP_YEAR,EXP_MONTH,PRICE,MATURITY,TTM
count,8.000000e+00,8,8.00000,8.000000,8.000000e+00,8.000000
mean,1.396224e+18,2014,7.50000,20.212500,1.407002e+18,0.341750
std,0.000000e+00,0,2.44949,1.113793,6.527919e+15,0.207062
min,1.396224e+18,2014,4.00000,17.850000,1.397779e+18,0.049000
25%,1.396224e+18,2014,5.75000,19.850000,1.402466e+18,0.198000
50%,1.396224e+18,2014,7.50000,20.550000,1.406851e+18,0.337000
75%,1.396224e+18,2014,9.25000,20.975000,1.411690e+18,0.490250
max,1.396224e+18,2014,11.00000,21.250000,1.416528e+18,0.644000
